In [6]:
from dotenv import load_dotenv
import random
from typing import TypedDict
from typing import Literal
import string
import xml.etree.ElementTree as ET

load_dotenv()

WINNING_VALUE = 256


# Class that keeps track of state for a single game of 2048
class TwentyFortyEightGame(TypedDict):
    id: str
    board: list[list[int | None]]


# Randomly populates a cell on the board with a 2 or 4
def populate_random_cell(game: TwentyFortyEightGame) -> None:
    all_clear_coordinates = [
        (i, j)
        for i in range(len(game["board"]))
        for j in range(len(game["board"][i]))
        if game["board"][i][j] is None
    ]
    random_clear_coordinates = random.choice(all_clear_coordinates)
    # 90% chance to populate a 2, 10% chance to populate a 4
    game["board"][random_clear_coordinates[0]][random_clear_coordinates[1]] = (
        2 if random.random() < 0.9 else 4
    )


# Generates a new game of 2048
def generate_game(board_length: int = 4) -> TwentyFortyEightGame:
    # random 6 character string
    id = "".join(random.choices(string.ascii_letters + string.digits, k=6))
    game = {
        "id": id,
        "board": [[None for _ in range(board_length)] for _ in range(board_length)],
    }

    # populate two random cells
    populate_random_cell(game)
    populate_random_cell(game)

    return game


# Renders the board in a human-readable format
def render_board(game: TwentyFortyEightGame) -> str:
    board = game["board"]
    # print something like this:
    # _    | 2    | _    | 4
    # 4    | 8    | 2    | 16
    # 16   | 32   | 64   | 128
    # _    | 2    | 2    | 4
    # where _ is an empty cell

    max_cell_width = max(
        [len(str(cell)) for row in board for cell in row if cell is not None]
    )

    board_str = ""
    for row in board:
        # pad the cells with spaces to make them the same width
        board_str += "|".join(
            [
                str(cell).rjust(max_cell_width)
                if cell is not None
                else "_".rjust(max_cell_width)
                for cell in row
            ]
        )
        board_str += "\n"
    return board_str


# condense, privileging matches at the start of the sequence
# sequences should be passed starting with cells that are the furthest in the direction in which the board is being condensed
def condense_sequence(sequence: list[int | None]) -> list[int | None]:
    condensed_sequence = []

    gapless_sequence = [cell for cell in sequence if cell is not None]

    i = 0
    while i < len(gapless_sequence):
        if (
            i + 1 < len(gapless_sequence)
            and gapless_sequence[i] == gapless_sequence[i + 1]
        ):
            condensed_sequence.append(gapless_sequence[i] * 2)
            i += 2
        else:
            condensed_sequence.append(gapless_sequence[i])
            i += 1

    # pad the sequence with None at the end
    return condensed_sequence + [None] * (4 - len(condensed_sequence))


# Condenses the board in a given direction
def condense_board(
    game: TwentyFortyEightGame, direction: Literal["left", "right", "up", "down"]
) -> None:
    if direction == "left":
        for row in game["board"]:
            condensed_row = condense_sequence(row)
            for i in range(len(row)):
                row[i] = condensed_row[i]

    if direction == "right":
        for row in game["board"]:
            reversed_row = row[::-1]
            # reverse the row before and after condensing
            condensed_row = condense_sequence(reversed_row)[::-1]
            for i in range(len(row)):
                row[i] = condensed_row[i]

    if direction == "up":
        for col_index in range(len(game["board"][0])):
            column = [row[col_index] for row in game["board"]]

            condensed_column = condense_sequence(column)
            for row_index in range(len(column)):
                game["board"][row_index][col_index] = condensed_column[row_index]

    if direction == "down":
        for col_index in range(len(game["board"][0])):
            column = [row[col_index] for row in game["board"]]
            reversed_column = column[::-1]
            condensed_column = condense_sequence(reversed_column)[::-1]
            for row_index in range(len(column)):
                game["board"][row_index][col_index] = condensed_column[row_index]


# Applies an agent move to the game board
def apply_agent_move(game: TwentyFortyEightGame, move_xml: str) -> None:
    direction = None
    # parse the move
    try:
        root = ET.fromstring(move_xml)
        direction = root.text
    except Exception as e:
        raise ValueError("Invalid xml")

    if direction not in ["left", "right", "up", "down"]:
        raise ValueError("Invalid direction")

    condense_board(game, direction)

    populate_random_cell(game)


# Returns the maximum cell value on the board
def max_cell_value(game: TwentyFortyEightGame) -> int:
    return max([cell for row in game["board"] for cell in row if cell is not None])


# Returns True if the game is finished
def check_game_finished(game: TwentyFortyEightGame) -> bool:
    if max_cell_value(game) >= WINNING_VALUE:
        return True

    # check if any cell is empty
    if any(cell is None for row in game["board"] for cell in row):
        return False

    return True


# Returns the sum of all the cell values on the board
def total_board_value(game: TwentyFortyEightGame) -> int:
    return sum([cell for row in game["board"] for cell in row if cell is not None])


In [7]:
import art
from art.local import LocalAPI
from dotenv import load_dotenv
from openpipe.client import AsyncOpenPipe
import random

load_dotenv()

random.seed(42)

# Initialize the server
api = LocalAPI()

REGISTER_TRAINABLE_MODEL = False
if REGISTER_TRAINABLE_MODEL:
    # Declare the model
    model = art.TrainableModel(
        name="012",
        project="2048-dev",
        base_model="Qwen/Qwen2.5-3B-Instruct",
        # To run on a T4, we need to override some config defaults.
        _internal_config=art.dev.InternalModelConfig(
            init_args=art.dev.InitArgs(
                max_seq_length=8192,
            ),
            engine_args=art.dev.EngineArgs(
                enforce_eager=True,
                gpu_memory_utilization=0.8,
                num_scheduler_steps=1,
            ),
        ),
    )
    await model.register(api)

In [8]:
import art

import openai
import time
import math
import requests


@art.retry(exceptions=(openai.LengthFinishReasonError, requests.ReadTimeout))
async def rollout(
    model: art.Model, step: int, is_validation: bool
) -> art.Trajectory:
    game = generate_game()

    move_number = 0

    trajectory = art.Trajectory(
        messages_and_choices=[
            {
                "role": "system",
                "content": "You are an excellent 2048 player. Always choose the move most likely to lead to combine cells to eventually reach the number 2048. Optional moves are 'left', 'right', 'up', 'down'. Return your move as an XML object with a single property 'move', like so: <move>left</move>",
            }
        ],
        reward=0,
    )

    while True:
        trajectory.messages_and_choices.append(
            {"role": "user", "content": render_board(game)}
        )

        requested_at = int(time.time() * 1000)
        messages = trajectory.messages()

        async def get_completion():
            client = model.openai_client()
            return await client.chat.completions.create(
                max_completion_tokens=128,
                messages=messages,
                model=model.name,
            )

        try:
            chat_completion = await get_completion()
            last_completion = chat_completion
        except openai.LengthFinishReasonError as e:
            raise e
        except Exception as e:
            print("caught exception generating chat completion", e)
            raise e

        try:
            if op_client.api_key:
                await op_client.report(
                    requested_at=requested_at,
                    received_at=int(time.time() * 1000),
                    req_payload={
                        "model": model.name,
                        "messages": messages,
                        "metadata": {
                            "game_id": game["id"],
                            "notebook-id": "2048",
                            "step": str(step),
                            "validation": str(is_validation),
                            "move_number": str(move_number),
                        },
                    },
                    resp_payload=chat_completion,
                    status_code=200,
                )
        except Exception as e:
            print(f"Error reporting to OpenPipe: {e}")

        choice = chat_completion.choices[0]
        content = choice.message.content
        assert isinstance(content, str)
        trajectory.messages_and_choices.append(choice)

        try:
            apply_agent_move(game, content)
            move_number += 1
        except ValueError:
            trajectory.reward = -1
            break

        if check_game_finished(game):
            max_value = max_cell_value(game)
            board_value = total_board_value(game)
            trajectory.metrics["max_value"] = max_value
            trajectory.metrics["board_value"] = board_value

            if max_value < WINNING_VALUE:
                # scale max value logarithmically between 0 for 2 and 1 for WINNING_VALUE
                max_value_reward = (math.log(max_value, 2) - 1) / (
                    math.log(WINNING_VALUE, 2) - 1
                )
                # scale board value logarithmically between 0 for 2 * 16 and 1 for WINNING_VALUE * 16
                board_value_reward = (math.log(board_value, 2) - 1) / (
                    math.log(WINNING_VALUE * 16, 2) - 1
                )
                # combine the two rewards, with max value having a higher weight
                trajectory.reward = max_value_reward + (board_value_reward * 0.2)
                trajectory.metrics["win"] = 0
            else:
                # double reward if the agent wins
                trajectory.reward = 2
                trajectory.metrics["win"] = 1
            break

    try:
        if op_client.api_key:
            await op_client.update_log_metadata(
                filters=[
                    {
                        "field": "completionId",
                        "equals": last_completion.id,
                    }
                ],
                metadata={
                    "reward": str(trajectory.reward),
                    "reward_assigned": "true",
                },
            )
    except Exception as e:
        print(f"Error updating log metadata: {e}")

    return trajectory

In [9]:
if REGISTER_TRAINABLE_MODEL:
    for i in range(await model.get_step(), 50):
        train_groups = await art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(
                    rollout(model, i, is_validation=False) for _ in range(4)
                )
                for _ in range(1)
            ),
            pbar_desc="gather",
            max_exceptions=1,
        )
        await model.delete_checkpoints()
        await model.train(
            train_groups,
            config=art.TrainConfig(learning_rate=3e-5),
            # Lowering the logprob_calculation_chunk_size is a memory saving measure
            # to allow longer sequences (up to 4096 tokens) to be processed on a T4.
            _config={"logprob_calculation_chunk_size": 8},
        )

In [12]:
import asyncio
import os


async def log_comparison_model(comparison_model: art.Model):
    trajectories = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(
                rollout(comparison_model, 0, is_validation=True) for _ in range(12)
            )
            for _ in range(1)
        ),
        pbar_desc=f"gather {comparison_model.name}",
        max_exceptions=1,
    )

    await comparison_model.log(
        trajectories,
        split="val",
    )

gpt_4o_mini = art.Model(
    name="gpt-4o-mini",
    project="2048-dev",
    inference_model_name="gpt-4o-mini",
    inference_base_url="https://api.openai.com/v1",
    inference_api_key=os.getenv("OPENAI_API_KEY"),
)
await gpt_4o_mini.register(api)

gpt_4o = art.Model(
    name="gpt-4o",
    project="2048-dev",
    inference_model_name="gpt-4o",
    inference_base_url="https://api.openai.com/v1",
    inference_api_key=os.getenv("OPENAI_API_KEY"),
)
await gpt_4o.register(api)

gpt_4_1 = art.Model(
    name="gpt-4.1",
    project="2048-dev",
    inference_model_name="gpt-4.1",
    inference_base_url="https://api.openai.com/v1",
    inference_api_key=os.getenv("OPENAI_API_KEY"),
)
await gpt_4_1.register(api)


# Optional logging client
op_client = AsyncOpenPipe()

promises = []

for comparison_model in [gpt_4o_mini, gpt_4o, gpt_4_1]:
    promises.append(log_comparison_model(comparison_model))

await asyncio.gather(*promises)


gather gpt-4o-mini:   0%|          | 0/12 [00:00<?, ?it/s]

gather gpt-4o:   0%|          | 0/12 [00:00<?, ?it/s]

gather gpt-4.1:   0%|          | 0/12 [00:00<?, ?it/s]

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-11' coro=<rollout() done, defined at /root/sky_workdir/src/art/utils/retry.py:74> exception=ValueError('In order to create an OpenAI client you must provide an `inference_api_key` and `inference_base_url`.')>
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "/root/sky_workdir/src/art/utils/retry.py", line 81, in async_wrapper
    return await func(*args, **kwargs)
  File "/tmp/ipykernel_11267/3596610620.py", line 50, in rollout
    raise e
  File "/tmp/ipykernel_11267/3596610620.py", line 44, in rollout
    chat_completion = await get_completion()
  File "/tmp/ipykernel_11267/3596610620.py", line 36, in get_completion
    client = model.openai_client()
  File "/root/sky_workdir/src/art/model.py", line 102, in openai_client
    raise ValueError(
ValueError: In order to create an OpenAI client you must prov

[None, None, None]

In [11]:
from art.utils.old_benchmarking.generate_comparison_table import generate_comparison_table
from art.utils.old_benchmarking.types import BenchmarkedModelKey

table = generate_comparison_table(
    project="2048-dev",
    benchmark_keys=[
        BenchmarkedModelKey("009", "train", [0, -2, -1]),
        BenchmarkedModelKey("gpt-4o-mini", "val"),
        BenchmarkedModelKey("gpt-4o", "val"),
    ],
    metrics=["reward", "max_value", "board_value"],
)

print(table.to_markdown())

FileNotFoundError: [Errno 2] No such file or directory: './.art/2048-dev/models/009/history.jsonl'

In [12]:
from art.utils.old_benchmarking.generate_line_graphs import generate_line_graphs
from art.utils.old_benchmarking.display_image_grid import display_image_grid
from art.utils.old_benchmarking.types import BenchmarkedModelKey

graph_image_paths = generate_line_graphs(
    project="2048-dev",
    line_graph_keys=[
        BenchmarkedModelKey("009", "train"),
        BenchmarkedModelKey("010", "train"),
        BenchmarkedModelKey("011", "train"),
    ],
    comparison_keys=[
        BenchmarkedModelKey("gpt-4o-mini", "val"),
        BenchmarkedModelKey("gpt-4o", "val"),
    ],
    metrics=["reward", "max_value", "board_value"],
)

display_image_grid(graph_image_paths)